<a href="https://colab.research.google.com/github/kgreed4/parkisons_exploration/blob/main/stand_to_sit/sts_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense

In [30]:
# Set the path to the csv
path = "/content/SitToStand_human_labels.csv"

# Read the csv into a dataframe
df = pd.read_csv(path)

# Display the first few rows of the dataframe
print(df.head())

   Transition ID  Participant ID number PD_or_C  sts_whole_episode_duration  \
0              1                    596      PD                    1.748000   
1              2                    596      PD                    2.085000   
2              3                    596      PD                    2.610000   
3              4                    596      PD                    2.478999   
4              5                    596      PD                    1.268000   

   sts_final_attempt_duration On_or_Off_medication DBS_state  \
0                    1.209000        On medication         -   
1                    1.076999        On medication         -   
2                    1.067999        On medication         -   
3                    1.233000        On medication         -   
4                    0.801000        On medication         -   

  Clinical_assessment STS_additional_features  \
0                 Yes                     NaN   
1                 Yes                     

In [31]:
# Create a neural network based on the df, where y = PD_or_C column

# Drop participant ID number
df = df.drop(columns=['Participant ID number'])

# Encode categorical variables
label_encoder = LabelEncoder()
df['PD_or_C'] = label_encoder.fit_transform(df['PD_or_C'])
df['On_or_Off_medication'] = label_encoder.fit_transform(df['On_or_Off_medication'])
df['STS_additional_features'] = label_encoder.fit_transform(df['STS_additional_features'])
df['DBS_state'] = label_encoder.fit_transform(df['DBS_state'])
df['Clinical_assessment'] = label_encoder.fit_transform(df['Clinical_assessment'])

In [33]:
# Check for missing values in each column
missing_values = df.isna().sum()

# Display columns with missing values
columns_with_missing_values = missing_values[missing_values > 0]
print("Columns with missing values:")
print(columns_with_missing_values)

Columns with missing values:
MDS-UPDRS_score_3.9 _arising_from_chair    7
dtype: int64


In [34]:
# Replace missing values
df['MDS-UPDRS_score_3.9 _arising_from_chair'].fillna(-1, inplace=True)

In [35]:
# Split data into features and target
X = df.drop(columns=['PD_or_C'])
y = df['PD_or_C']

# Standardize features
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Split data into training, val, and testing sets into a 70, 15, 15 split
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.3, random_state=42)
X_test, X_val, y_test, y_val = train_test_split(X_val, y_val, test_size=0.15, random_state=42)

# Define the neural network architecture
model = Sequential([
    Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    Dense(32, activation='relu'),
    Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_val, y_val))

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Test Loss: {loss}, Test Accuracy: {accuracy}')

Epoch 1/10
9/9 [==============================] - 1s 52ms/step - loss: 0.6059 - accuracy: 0.7270 - val_loss: 0.5844 - val_accuracy: 0.7895
Epoch 2/10
9/9 [==============================] - 0s 8ms/step - loss: 0.5281 - accuracy: 0.8333 - val_loss: 0.5260 - val_accuracy: 0.8421
Epoch 3/10
9/9 [==============================] - 0s 9ms/step - loss: 0.4613 - accuracy: 0.8936 - val_loss: 0.4656 - val_accuracy: 0.9474
Epoch 4/10
9/9 [==============================] - 0s 10ms/step - loss: 0.4058 - accuracy: 0.9255 - val_loss: 0.4113 - val_accuracy: 0.9474
Epoch 5/10
9/9 [==============================] - 0s 10ms/step - loss: 0.3569 - accuracy: 0.9468 - val_loss: 0.3624 - val_accuracy: 0.9474
Epoch 6/10
9/9 [==============================] - 0s 10ms/step - loss: 0.3128 - accuracy: 0.9574 - val_loss: 0.3167 - val_accuracy: 0.9474
Epoch 7/10
9/9 [==============================] - 0s 8ms/step - loss: 0.2732 - accuracy: 0.9645 - val_loss: 0.2782 - val_accuracy: 0.9474
Epoch 8/10
9/9 [==============